In [ ]:
import requests
import pandas as pd
import json
import os
import time

# Informations d'authentification pour accéder à l'API France Travail
client_id = "PAR_jobmarket_7e4143861bb41cbd5774fa245747f8cf7734fe1b1d2c2b86da829288d3325443"
client_secret = "9a7ec985f56098e8ea8040cbcd6e5efc8f37400c90c3884db5349773088a7b27"

# URL de l'API pour obtenir le token d'authentification
url_token = "https://entreprise.francetravail.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"

# URL de l'API pour récupérer les offres d'emploi
url_offres = "https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"

# Payload pour obtenir le token d'accès
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': "o2dsoffre api_offresdemploiv2",
    'expires_in': '1499'  # Durée de validité du token en secondes (1499 secondes)
}

# Variables pour stocker le token et son expiration
access_token = None
token_expiration_time = 0

def get_new_token():
    """
    Cette fonction obtient un nouveau token d'accès depuis l'API.
    Elle met à jour les variables globales access_token et token_expiration_time.
    """
    global access_token, token_expiration_time
    auth_response = requests.post(url_token, data=payload)
    if auth_response.status_code == 200:
        token_data = auth_response.json()
        access_token = token_data.get("access_token")
        expires_in = token_data.get("expires_in", 0)
        token_expiration_time = time.time() + expires_in
        print("Nouveau token obtenu:", access_token)
        print("Durée de validité du token (en secondes):", expires_in)
    else:
        print("Erreur lors de l'authentification:", auth_response.status_code)
        print("Message d'erreur:", auth_response.text)

# Obtenir un token initial pour accéder à l'API
get_new_token()

def extract_fields(offer):
    """
    Cette fonction extrait les champs imbriqués d'une offre d'emploi et les renvoie sous forme de dictionnaire.
    """
    try:
        fields = {
            'id': offer.get('id'),
            'intitule': offer.get('intitule'),
            'description': offer.get('description'),
            'dateCreation': offer.get('dateCreation'),
            'dateActualisation': offer.get('dateActualisation'),
            'lieuTravail_libelle': offer['lieuTravail'].get('libelle') if offer.get('lieuTravail') else None,
            'lieuTravail_codePostal': offer['lieuTravail'].get('codePostal') if offer.get('lieuTravail') else None,
            'lieuTravail_commune': offer['lieuTravail'].get('commune') if offer.get('lieuTravail') else None,
            'appellationlibelle': offer.get('appellationlibelle'),
            'typeContrat': offer.get('typeContrat'),
            'typeContratLibelle': offer.get('typeContratLibelle'),
            'natureContrat': offer.get('natureContrat'),
            'salaire_libelle': offer['salaire'].get('libelle') if offer.get('salaire') else None,
            'salaire_complement1': offer['salaire'].get('complement1') if offer.get('salaire') else None,
            'salaire_complement2': offer['salaire'].get('complement2') if offer.get('salaire') else None,
            'dureeTravailLibelle': offer.get('dureeTravailLibelle'),
            'dureeTravailLibelleConverti': offer.get('dureeTravailLibelleConverti'),
            'secteurActivite': offer.get('secteurActivite'),
            'secteurActiviteLibelle': offer.get('secteurActiviteLibelle'),
            'origineOffre_origine': offer['origineOffre'].get('origine') if offer.get('origineOffre') else None,
            'origineOffre_urlOrigine': offer['origineOffre'].get('urlOrigine') if offer.get('origineOffre') else None,
            'entreprise_nom': offer['entreprise'].get('nom') if offer.get('entreprise') else None,
            'contact_nom': offer['contact'].get('nom') if offer.get('contact') else None,
            'contact_coordonnees1': offer['contact'].get('coordonnees1') if offer.get('contact') else None,
            'contact_courriel': offer['contact'].get('courriel') if offer.get('contact') else None
        }
        return fields
    except Exception as e:
        print(f"Erreur lors de l'extraction des champs de l'offre ID {offer.get('id', 'inconnu')}: {e}")
        return {}

def make_request_with_retry(url, headers):
    """
    Cette fonction effectue une requête à l'API avec gestion de l'attente exponentielle en cas d'erreur 429 (trop de requêtes).
    """
    max_retries = 5
    wait_time = 1  # Temps d'attente initial en secondes
    for attempt in range(max_retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 206:
            return response
        elif response.status_code == 429:  # Code d'état pour trop de requêtes
            print(f"Trop de requêtes. Attente de {wait_time} secondes avant la tentative {attempt + 1}.")
            time.sleep(wait_time)
            wait_time *= 2  # Double le temps d'attente pour la prochaine tentative
        else:
            print(f"Erreur lors de la requête: {response.status_code}. Réponse: {response.text}")
            break
    return None

# Définir les valeurs de la plage pour la pagination
debut = 0
fin = 99  # Récupérer 100 éléments à la fois

all_offres = []
seen_ids = set()  # Ensemble pour stocker les IDs des offres déjà vus

max_attempts = 30
attempts = 0

while True:
    # Vérifier si le token est encore valide, sinon obtenir un nouveau token
    if time.time() >= token_expiration_time:
        if attempts >= max_attempts:
            print("Le nombre maximum de tentatives de renouvellement du token a été atteint.")
            break
        get_new_token()
        attempts += 1

    # Configuration des en-têtes pour la requête d'offres
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {access_token}",
        "Range": f"offres={debut}-{fin}"
    }

    # Envoyer la demande avec l'en-tête de plage
    response = make_request_with_retry(url_offres, headers)

    # Vérification de la réponse
    if response and response.status_code == 206:  # 206 indique une réponse partielle
        data_offres = response.json()  # Récupérer la réponse JSON

        # Extraire les données JSON
        resultats = data_offres["resultats"]

        # Traiter chaque offre d'emploi récupérée
        for offer in resultats:
            offer_id = offer.get('id')
            if offer_id not in seen_ids:
                seen_ids.add(offer_id)
                data_list = extract_fields(offer)
                all_offres.append(data_list)

        # Si le nombre d'offres récupérées est inférieur à la plage spécifiée, toutes les offres ont été récupérées
        if len(resultats) < (fin - debut + 1):
            break

        # Mettre à jour les valeurs de début et de fin pour la prochaine demande
        debut += 100
        fin += 100
    else:
        print("Erreur lors de la récupération des données d'emploi.")
        break

# Définir le chemin du fichier dans le sous-dossier Projet_api_france_30_05_24
output_directory = 'Projet_api_france_30_05_24'
os.makedirs(output_directory, exist_ok=True)
filename = os.path.join(output_directory, 'offres_emploi.json')

# Enregistrer les données dans un fichier JSON
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(all_offres, f, ensure_ascii=False, indent=4)

print("Données enregistrées dans le fichier offres_emploi.json")

# Afficher la taille du fichier
file_size = os.path.getsize(filename)
print(f"Taille du fichier {filename} : {file_size} octets")

# Afficher le nombre de documents récupérés
print(f"Nombre de documents récupérés data_list: {len(all_offres)}")
